In [1]:
##clean code for undeepVO
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,Conv2DTranspose,UpSampling2D,MaxPooling2D,Concatenate,BatchNormalization,Flatten
from tensorflow.keras.layers import Dense,Layer
import numpy as np
from losses import *
from nets.dep_network import depth_estimation

In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate=.001)
@tf.function
def train_step(image_left,image_right):
    with tf.GradientTape(persistent=True) as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        dcx7_right,dep_right = de(image_right)
        dcx7_left,dep_left = de(image_left)
        loss_depth = dept_est_loss(dep_left,dep_right,image_left,image_right)
        #tf.print("loss_depth",loss_depth)
        ##not using pose estimation currently because of tensorflow 2.0
        #t_left,rot_left = pe(image_left)
        #t_right,rot_right = pe(image_right)
        #loss_pose = pose_est_loss(rot_right,rot_left,t_left,t_right)
        #tf.print("loss_pose",loss_pose)

    gradients_de = tape.gradient(loss_depth, de.trainable_variables)

    #  gradients_pe = tape.gradient(loss_pose, pe.trainable_variables)
    optimizer.apply_gradients(zip(gradients_de, de.trainable_variables))
    #  optimizer.apply_gradients(zip(gradients_pe, pe.trainable_variables))
    return loss_depth#,loss_pose

In [10]:
EPOCHS = 5
train_ds = []

de = depth_estimation()

for i in range(5):
    #images_left = np.random.randint(0,255,(3,256,256,3)).astype(np.float32)
    #images_right = np.random.randint(0,255,(3,256,256,3*2)).astype(np.float32)
    images_left = tf.ones((3,256,256,3*2),dtype=tf.float32)*255
    images_right = tf.ones((3,256,256,3*2),dtype=tf.float32)*(255+2)
    train_ds.append([images_left,images_right])

#images_left = tf.ones((1,256,256,3*2),dtype=tf.float32)*(0)
#images_right = tf.ones((1,256,256,3*2),dtype=tf.float32)*(1)

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    for images_left,images_right in train_ds:
        ld = train_step(images_left,images_right)
    print("epochs : ",epoch)
    print("loss depth",ld.numpy())
    #print("loss pose",lp)

InvalidArgumentError:  indices[184320] = -2147296598 is not in [0, 199692)
	 [[node GatherV2 (defined at /home/roboticist/Documents/Swaayatt/swaayatt_optical_flow/github_dir/Implementation-of-UnDeepVO/utils.py:46) ]] [Op:__inference_train_step_11227]

Errors may have originated from an input operation.
Input Source operations connected to node GatherV2:
 add_10 (defined at /home/roboticist/Documents/Swaayatt/swaayatt_optical_flow/github_dir/Implementation-of-UnDeepVO/utils.py:41)	
 Reshape_8 (defined at /home/roboticist/Documents/Swaayatt/swaayatt_optical_flow/github_dir/Implementation-of-UnDeepVO/utils.py:44)

Function call stack:
train_step


In [ ]:
"""
checking the depth estimation and loss is working
_,disp_left = de(images_left)
print(disp_left[:,:,:,0])
de1 = depth_estimation()
#_,disp_left1 = de(images_left)
dcx7_right,dep_right = de1(images_right)
dcx7_left,dep_left = de1(images_left)
loss_depth = dept_est_loss(dep_left,dep_right,images_left,images_right)
print(loss_depth.numpy())
dcx7_right,dep_right = de(images_right)
dcx7_left,dep_left = de(images_left)
loss_depth = dept_est_loss(dep_left,dep_right,images_left,images_right)
print(loss_depth.numpy())
"""